# $Importation$ $des$ $bibliothèques$

In [1]:
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
import os
from scipy.fftpack import dct, idct
import matplotlib.pyplot as plt
import re

/home/issa-sangare/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# $Données/Informations$ $utilisées$ $pour$ $le$ $programme$

In [2]:
path = '3.jpeg'
path1 = 'file.txt'
path2 = 'reconstiued3.jpg'
num_rows = 4
num_colors = 64
num_blocs = 16

# $Caratéristiques$ $de$ $l'image$

In [3]:
def details(path):
    img = Image.open(path)
    size = img.size
    format = img.format
    mode = img.mode
    resolution = img.info.get('dpi')
    definition = size[0] * size[1]
    definition2 = size
    
    if resolution is not None:
        size = (size[0]/ resolution[0], size[1] / resolution[1])
    else:
        size = (0, 0)  # or any default resolution you want to use
    
    poids = os.path.getsize(path) / (1000.0)  # Convert to kilobytes
    
    if mode == 'RGB':
        trsc = definition * 3 * img.bits
    else:
        trsc = definition * 1 * img.bits
    
    trsc /= (1024 * 8)
    
    taux_compression = 100 * (1 - (poids/trsc))
        
    return size, format, mode, resolution, definition, definition2, poids, trsc, taux_compression

In [4]:
# Exemple d'utilisation
result = details(path)
print("Size (inch):", result[0])
print("Format:", result[1])
print("Mode:", result[2])
print("Resolution:", result[3])
print("Definition (pixels):", result[4])
print("Définition (L * C):", result[5])
print("Taille en mémoire avec compression:", result[6], "KB")
print("Taille réelle sans compression:", result[7]/1024, "MB")
print("Taux de compression: ", result[8], "%")

Size (inch): (4.2, 2.36)
Format: JPEG
Mode: RGB
Resolution: (300, 300)
Definition (pixels): 892080
Définition (L * C): (1260, 708)
Taille en mémoire avec compression: 113.617 KB
Taille réelle sans compression: 2.5522613525390625 MB
Taux de compression:  95.65271395689476 %


# $Affichage$ $de$ $la$ $palette$ $de$ $couleurs$ $et$ $des$ $images$ $avant$ $et$ $après$ $application$ $de$ $la$ $palette$ $de$ $couleurs$

In [5]:
def display_image_with_palette(image, color_palette, palette_indices, num_rows):
    num_colors = len(color_palette)
    num_cols = num_colors // num_rows

    image_with_palette = color_palette[palette_indices]

    plt.figure(figsize=(30, 15))
    for i, color in enumerate(color_palette):
        plt.subplot(num_rows, num_cols, i + 1)
        color_patch = np.zeros((100, 100, 3), dtype=np.uint8)
        color_patch[:, :] = color
        plt.imshow(color_patch)
        plt.title(f'Couleur {i+1}')
        plt.axis('off')
    plt.suptitle('Palette de couleurs', fontsize=16)
    plt.tight_layout()
    plt.show()

    fig, axs = plt.subplots(1, 2, figsize=(30, 15))
    fig.subplots_adjust(wspace=0.01)

    axs[0].imshow(image)
    axs[0].set_title('Image originale')
    axs[0].axis('off')

    axs[1].imshow(image_with_palette)
    axs[1].set_title(f'Image avec la palette de {num_colors} couleurs')
    axs[1].axis('off')

    fig.tight_layout()

    plt.show()

# $Affichage$ $de$ $deux$ $images$ $quelconques$ $côte$ $à$ $côte$

In [6]:
def plot_images(reconstructed_image, image1, num_colors):
    fig, axs = plt.subplots(1, 2, figsize=(30, 15))
    fig.subplots_adjust(wspace=0.01)

    axs[0].imshow(reconstructed_image)
    axs[0].set_title('Image reconstruite')
    axs[0].axis('off')

    axs[1].imshow(image1)
    axs[1].set_title(f'Image avec la palette de {num_colors} couleurs (median-cut)')
    axs[1].axis('off')

    fig.tight_layout()

    plt.show()

## $Mean$ $Square$ $Error$ $(MSE):$ $Erreur$ $Quadratique$ $Moyenne$

In [7]:
def MSE(image_d_origine, image_traitee):
    # Get the dimensions of the images
    L, C = np.array(image_d_origine).shape[:2]  # Assuming it's a 2D image, getting rows and columns
    image_traitee = np.array(image_traitee).reshape(np.array(image_d_origine).shape)
    # Convert the images to NumPy arrays
    origine = np.array(image_d_origine)
    traitee = np.array(image_traitee)

    # Calculate the sum of squared differences
    somme_differences_carrees = np.sum((origine - traitee) ** 2)

    # Calculate (1 / (L * C)) * somme
    resultat = (1 / (L * C)) * somme_differences_carrees

    return resultat

# $Taux$ $de$ $compression$

In [8]:
def taux_compression(img_path, compressed_file_path):
    img = Image.open(img_path)
    
    if img.mode == 'RGB':
        pixel_size = 3  # Nombre de canaux de couleur (RGB)
    else:
        pixel_size = 1  # Pour d'autres modes de couleur
    
    if img.bits == 8:  # Taille en bits par pixel
        pixel_bits = 8
    else:
        pixel_bits = 16  # Peut être ajusté selon les besoins
    
    img_data_size = img.size[0] * img.size[1] * pixel_size * pixel_bits  # Taille en bits de l'image
    
    # Lecture des données compressées à partir du fichier .irm
    with open(compressed_file_path, 'rb') as f:
        compressed_data = f.read()
    
    compressed_data_size = len(compressed_data) * 8  # Taille des données compressées en bits
    
    compression_ratio = img_data_size / compressed_data_size  # Ratio de compression
    
    return compression_ratio

# (1). $Création$ $de$ $la$ $palette$ $de$ $couleurs$

## (a). $Sens$ $allée$

### $k-means$ $clustering$

In [9]:
def kmeans_clustering_palette(image, num_colors):
    # Convertir l'image en un tableau 2D de pixels
    pixels = np.reshape(image, (-1, 3))  # (nombre de pixels, 3 canaux de couleur)

    # Appliquer l'algorithme de k-means clustering
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(pixels)

    # Obtenir les centres des clusters (couleurs dominantes)
    color_palette = kmeans.cluster_centers_.astype(int)

    # Associer chaque pixel à l'indice de couleur dans la palette
    labels = kmeans.predict(pixels)

    # Reformater les indices des couleurs dans la palette selon la forme de l'image originale
    palette_indices = np.reshape(labels, np.array(image).shape[:2])

    return color_palette, palette_indices

### $Median-cut$

In [10]:
def median_cut_palette(image, num_colors):
    # Convertir l'image en un tableau 2D de pixels
    pixels = np.reshape(image, (-1, 3))

    # Initialiser la liste des cubes de couleur avec le cube contenant tous les pixels
    cubes = [pixels]

    # Répéter jusqu'à ce que le nombre de cubes atteigne le nombre de couleurs souhaité
    while len(cubes) < num_colors:
        # Sélectionner le cube le plus grand
        largest_cube_index = np.argmax([cube.shape[0] for cube in cubes])
        largest_cube = cubes.pop(largest_cube_index)

        # Trouver l'axe dominant pour diviser le cube
        axis = np.argmax(np.max(largest_cube, axis=0) - np.min(largest_cube, axis=0))

        # Trier les pixels du cube le long de l'axe dominant
        sorted_cube = largest_cube[largest_cube[:, axis].argsort()]

        # Diviser le cube en deux parties égales
        split_index = len(sorted_cube) // 2
        cube1 = sorted_cube[:split_index]
        cube2 = sorted_cube[split_index:]

        # Ajouter les deux nouveaux cubes à la liste
        cubes.append(cube1)
        cubes.append(cube2)

    # Calculer les couleurs moyennes pour chaque cube
    color_palette = [np.mean(cube, axis=0) for cube in cubes]
    color_palette = np.array(color_palette).astype(np.uint8)

    # Calculer les indices des couleurs dans la palette pour chaque pixel de l'image
    palette_indices = np.zeros(len(pixels), dtype=int)
    for i, pixel in enumerate(pixels):
        distances = np.linalg.norm(color_palette - pixel, axis=1)
        palette_indices[i] = np.argmin(distances)

    palette_indices = np.reshape(palette_indices, np.array(image).shape[:2])

    return color_palette, palette_indices

### $Octree$

In [11]:
def octree_palette(image, num_colors):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
        
    img_rgb = image.convert("RGB")
    img_with_palette = img_rgb.quantize(colors=num_colors, method=Image.FASTOCTREE)
    palette = img_with_palette.getpalette()[:num_colors * 3]
    palette = np.array(palette).reshape(-1, 3)
    indices = np.array(img_with_palette)

    # Retourner la palette de couleurs et leurs indices
    return palette, indices

## (b). $Sens$ $retour$

### $k-means$ $clustering$

In [12]:
def inverse_kmeans_clustering_palette(color_palette, palette_indices):
    # Récupérer les dimensions de l'image à partir des indices de palette
    height, width = palette_indices.shape

    # Initialiser une image vide avec les dimensions récupérées
    reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Remplir l'image reconstruite avec les couleurs de la palette
    for i in range(height):
        for j in range(width):
            color_index = palette_indices[i, j]
            reconstructed_image[i, j] = color_palette[color_index]

    return reconstructed_image

### $Median-cut$

In [13]:
def inverse_median_cut_palette(color_palette, palette_indices):
    # Récupérer les dimensions de l'image à partir des indices de palette
    height, width = palette_indices.shape

    # Initialiser une image vide avec les dimensions récupérées
    reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Remplir l'image reconstruite avec les couleurs de la palette
    for i in range(height):
        for j in range(width):
            color_index = palette_indices[i, j]
            reconstructed_image[i, j] = color_palette[color_index]
  
    return reconstructed_image

### $Octree$

In [14]:
def inverse_octree_palette(color_palette, palette_indices):
    color_palette = color_palette.reshape(-1)
    color_palette = [(int(color_palette[i]), int(color_palette[i + 1]), int(color_palette[i + 2])) for i in range(0, len(color_palette), 3)]
    reconstructed_image = Image.new("RGB", (palette_indices.shape[1], palette_indices.shape[0]))
    reconstructed_image.putdata([color_palette[idx] for idx in palette_indices.flatten()])
  
    return reconstructed_image

# (2). $Mappage$ $des$ $pixels$

## (a). $Sens$ $"allée"$

In [15]:
def map_to_palette(image, color_palette):
    # Assurez-vous que l'image est sous forme de tableau numpy
    image = np.array(image)
    
    # Redimensionner les pixels en une matrice (nombre de pixels, 3 canaux de couleur)
    pixels = np.reshape(image, (-1, 3))  

    # Calculer la distance de chaque pixel à chaque couleur de la palette
    distances = np.linalg.norm(pixels[:, np.newaxis] - color_palette, axis=2)

    # Obtenir l'indice de la couleur la plus proche pour chaque pixel
    indices = np.argmin(distances, axis=1)

    # Reformater les indices des couleurs selon la forme de l'image originale
    mapped_indices = np.reshape(indices, image.shape[:2])

    return np.array(mapped_indices)

## (b). $Sens$ $"retour"$

In [16]:
def inverse_map_to_palette(mapped_indices, color_palette):
    # Obtenir les dimensions de l'image reconstruite
    height, width = mapped_indices.shape

    # Créer une image vide
    reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Pour chaque pixel, assigner la couleur correspondante de la palette
    for i in range(height):
        for j in range(width):
            color_index = mapped_indices[i, j]
            reconstructed_image[i, j] = color_palette[color_index]

    return Image.fromarray(reconstructed_image)

# (3). $Subdivision$ $en$ $blocs$

## (a). $Sens$ $"allée"$

In [17]:
def subdivision2D(matrice, taille_blocs):
    # Dimensions de la matrice initiale
    lignes, colonnes = matrice.shape
    
    # Dimensions des blocs
    blocs_lignes, blocs_colonnes = (taille_blocs, taille_blocs)
    
    # Calcul des dimensions des blocs avec zéros ajoutés
    new_blocs_lignes = (lignes + blocs_lignes - 1) // blocs_lignes
    new_blocs_colonnes = (colonnes + blocs_colonnes - 1) // blocs_colonnes
    
    # Initialisation de la matrice des sous-matrices
    sous_matrices = np.zeros((new_blocs_lignes, new_blocs_colonnes, blocs_lignes, blocs_colonnes))
    
    # Remplissage de la matrice des sous-matrices
    for i in range(new_blocs_lignes):
        for j in range(new_blocs_colonnes):
            sous_matrices[i, j, :min(blocs_lignes, lignes - i*blocs_lignes), :min(blocs_colonnes, colonnes - j*blocs_colonnes)] = \
                matrice[i*blocs_lignes:(i+1)*blocs_lignes, j*blocs_colonnes:(j+1)*blocs_colonnes]
    
    # Convertir la matrice de sous-matrices en une liste de matrices 2D
    liste_matrices = []
    for i in range(new_blocs_lignes):
        for j in range(new_blocs_colonnes):
            liste_matrices.append(sous_matrices[i, j])
    
    return liste_matrices, (lignes, colonnes)

## (b). $Sens$ $"retour"$

In [18]:
def reconstitution2D(liste_matrices, dimensions):# Récupération des dimensions de la matrice initiale
    lignes, colonnes = dimensions
    
    # Initialisation de la matrice résultante avec des zéros
    matrice_resultante = np.zeros((lignes, colonnes))
    
    # Dimensions des blocs
    taille_blocs = liste_matrices[0].shape
    
    # Nombre de blocs
    nb_blocs_lignes = (lignes + taille_blocs[0] - 1) // taille_blocs[0]
    nb_blocs_colonnes = (colonnes + taille_blocs[1] - 1) // taille_blocs[1]
    
    # Recombinaison des sous-matrices dans la matrice résultante
    for i in range(nb_blocs_lignes):
        for j in range(nb_blocs_colonnes):
            matrice_resultante[i*taille_blocs[0]:(i+1)*taille_blocs[0], j*taille_blocs[1]:(j+1)*taille_blocs[1]] = \
                liste_matrices[i*nb_blocs_colonnes + j][:min(taille_blocs[0], lignes - i*taille_blocs[0]), :min(taille_blocs[1], colonnes - j*taille_blocs[1])]
    
    return matrice_resultante

# (4). $Sous$ $échantillonnage$

## (a). $Sens$ $"allée"$

### $4:2:0$

In [19]:
def sous_echantillonnage_4_2_0(image):
    # Sous-échantillonnage de l'image
    sub_image = image[::2, ::2]
    
    return sub_image

### $4:2:2$

In [20]:
def sous_echantillonnage_4_2_2(image):
    # Sous-échantillonnage de l'image
    sub_image = image[::1, ::2]
    
    return sub_image

### $4:4:4$

In [21]:
def sous_echantillonnage_4_4_4(image):
    # La fonction ne fait rien car il n'y a pas de sous-échantillonnage pour le 4:4:4
    return image.copy() 

## (b). $Sens$ $"retour"$

### $4:2:0$

In [22]:
def inverse_sous_echantillonnage_4_2_0(imSub):
    # Répétition des pixels pour restaurer la résolution horizontale
    H, W = imSub.shape
    res_h = np.repeat(imSub, 2, axis=1)
    
    # Répétition des pixels pour restaurer la résolution verticale
    res_v = np.repeat(res_h, 2, axis=0)
    
    return res_v

### $4:2:2$

In [23]:
def inverse_sous_echantillonnage_4_2_2(imSub):
    # Répétition des colonnes pour restaurer les canaux de chrominance
    res_h = np.repeat(imSub, 2, axis=1)
    
    return res_h

### $4:4:4$

In [24]:
def inverse_sous_echantillonnage_4_4_4(imSub):
    # L'inverse du sous-échantillonnage 4:4:4 est simplement une copie de l'image d'entrée
    return imSub.copy()

# (5). $Transformée$ $en$ $cosinus$ $discrète:$ $application$ $de$ $la$ $DCT$

## (a). $Sens$ $"allée"$

In [25]:
def apply_dct(matrix):
    # Appliquer la DCT à chaque matrice
    dct_matrix = dct(dct(matrix.T, norm='ortho').T, norm='ortho')
    
    return np.array(dct_matrix)

## (b). $Sens$ $"retour"$

In [26]:
def inverse_dct(dct_matrix):
    # Appliquer l'inverse de la DCT à chaque matrice
    inverse_dct_matrix = idct(idct(dct_matrix.T, norm='ortho').T, norm='ortho')
    
    return np.array(inverse_dct_matrix)

# (6). $Quantification$

## (a). $Sens$ $"allée"$

### $Valeur$ $minimale$

In [27]:
def quantization_min(matrix, epsilon=1e-10):
    # Déterminer la valeur minimale dans la matrice
    min_value = np.min(matrix)
    
    # Vérifier si la valeur minimale est proche de zéro avant la division
    if np.abs(min_value) > epsilon:
        # Quantification en utilisant la valeur minimale comme facteur de quantification
        quantized_dct = np.round(matrix / min_value)
    else:
        # Division par epsilon si la valeur minimale est proche de zéro
        quantized_dct = np.round(matrix / epsilon)
        
    if np.all(quantized_dct == 0):
        quantized_dct[0, 0] = 1
    
    return quantized_dct, min_value

### $Valeur$ $moyenne$

In [28]:
def quantization_mean(matrix, epsilon=1e-10):
    # Déterminer la valeur minimale et maximale dans la matrice
    min_value = np.min(matrix)
    max_value = np.max(matrix)
    
    # Calculer la moyenne des valeurs
    mean_value = (max_value + min_value) / 2
    
    # Vérifier si la valeur moyenne est proche de zéro avant la division
    if np.abs(mean_value) > epsilon:
        # Quantification en utilisant la valeur moyenne comme facteur de quantification
        quantized_dct = np.round(matrix / mean_value)
    else:
        # Division par epsilon si la valeur moyenne est proche de zéro
        quantized_dct = np.round(matrix / epsilon)
        
    if np.all(quantized_dct == 0):
        quantized_dct[0, 0] = 1
    
    return quantized_dct, mean_value

### $Valeur$ $maximale$

In [29]:
def quantization_max(matrix, epsilon=1e-10):
    # Déterminer la valeur maximale dans la matrice
    max_value = np.max(matrix)
    
    # Vérifier si la valeur maximale est proche de zéro avant la division
    if np.abs(max_value) > epsilon:
        # Quantification en utilisant la valeur maximale comme facteur de quantification
        quantized_dct = np.round(matrix / max_value)
    else:
        # Division par epsilon si la valeur maximale est proche de zéro
        quantized_dct = np.round(matrix / epsilon)
        
    if np.all(quantized_dct == 0):
        quantized_dct[0, 0] = 1
    
    return quantized_dct, max_value

## (b). $Sens$ $"retour"$

### $Valeur$ $minimale$

In [30]:
def dequantization_min(quantized_matrix, min_value):
    return np.array(quantized_matrix * min_value, dtype=np.float64)

### $Vaeur$ $moyenne$

In [31]:
def dequantization_mean(quantized_matrix, mean_value):
    return np.array(quantized_matrix * mean_value, dtype=np.float64)

### $Valeur$ $maximale$

In [32]:
def dequantization_max(quantized_matrix, max_value):
    return np.array(quantized_matrix * max_value, dtype=np.float64)

# (7). $Vectorisation$

## (a). $Sens$ $"allée"$

### $En$ $ligne$

In [33]:
def ligne_scan(matrice):
  shape = matrice.shape
  vecteur = matrice.flatten()
  return np.array(vecteur, dtype=np.int8), shape

### $En$ $colonne$

In [34]:
def colonne_scan(matrice):
    shape = matrice.shape
    vecteur = np.ravel(matrice, order = 'F')
    return np.array(vecteur, dtype=np.int8), shape

### $En$ $zigzag$

In [35]:
def zigzag_scan(matrice):
  shape = matrice.shape
  vecteur = np.concatenate([np.diagonal(matrice[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - matrice.shape[0], matrice.shape[0])])
  return np.array(vecteur, dtype=np.int8), shape

## (b). $Sens$ $"retour"$

### $En$ $ligne$

In [36]:
def inverse_ligne_scan(vecteur, shape):
  mat = np.array(vecteur).reshape((shape[0], shape[1]))
  return mat

### $En$ $colonne$

In [37]:
def inverse_colonne_scan(vecteur, shape):
    mat = np.array(vecteur).reshape((shape[0], shape[1]))
    return mat.T

### $En$ $zigzag$

In [38]:
def inverse_zigzag_scan(vector, shape):
    rows, cols = shape
    mat = np.array([[None] * cols for _ in range(rows)])
    row, col = 0, 0
    direction = 1

    for i in range(rows * cols):
        mat[row][col] = vector[i]
        if direction == 1:
            if col == cols - 1:
                row += 1
                direction = -1
            elif row == 0:
                col += 1
                direction = -1
            else:
                row -= 1
                col += 1
        else:
            if row == rows - 1:
                col += 1
                direction = 1
            elif col == 0:
                row += 1
                direction = 1
            else:
                row += 1
                col -= 1

    #return np.array(mat, dtype=np.uint8)
    return mat

# (8). $Codage$ $de$ $Huffman$

## (-). $Fonctions$ $intermédiaires$ $(propres$ $à$ $Huffman)$

### $Extraction$ $des$ $caractères$ $et$ $leur$ $fréquence$ $d'apparition$

In [39]:
def ext_chr(chaine):
    occ = {c: chaine.count(c) for c in set(chaine)}
    return sorted(occ.items(), key=lambda x: x[1], reverse=True)

### $Dictionnaire$ $de$ $Huffman$

In [40]:
def huffman_dictionnary_code(node, binString=''):
    if isinstance(node, str):
        return {node: binString}
    (l, r) = node
    d = {}
    d.update(huffman_dictionnary_code(l, binString + '0'))
    d.update(huffman_dictionnary_code(r, binString + '1'))
    return d

### $Arbre$ $de$ $Huffman$

In [41]:
def huffman_tree(chaine):
    nodes = ext_chr(chaine)
    
    while len(nodes) > 1:
        (key1, c1) = nodes.pop()
        (key2, c2) = nodes.pop()
        node = (key1, key2)
        nodes.append((node, c1 + c2))
        nodes.sort(key=lambda x: x[1], reverse=True)
        
    return nodes[0][0]

## (-). $Fonctions$ $intermédiaires$ $(impropres)$

### $Convertion$ $du$ $type$ $des$ $valeurs$ $du$ $vecteur$ $en$ $entier$ $non$ $signé$ $sur$ $8$ $bits$ $et$ $convertion$ $en$ $une$ $chaine$ $de$ $caractères$

#### (a). $Sens$ $"allée"$

In [42]:
def convert_and_concatenate(vector):
    # Conversion du vecteur en une liste de nombres non signés
    unsigned_vector = vector.astype(np.uint8) 
    
    # Obtention de la taille de chaque élément de la liste
    element_size = [len(str(i)) for i in unsigned_vector.tolist()]

    # Concaténation des nombres non signés en une liste unique
    concatenated_list = unsigned_vector.flatten().tolist()
    string = ''.join(map(str, concatenated_list))

    return string, element_size

#### (b). $Sens$ $"retour"$

In [43]:
def inverse_convert_and_concatenate(string, element_size):
    # Vérification que les arguments sont valides
    if not isinstance(string, str) or not isinstance(element_size, list):
        raise ValueError("Les arguments doivent être une chaîne de caractères et une liste.")

    # Convertir la chaîne en une liste de nombres non signés en utilisant les tailles d'éléments
    unsigned_vector = []
    start = 0
    for size in element_size:
        unsigned_vector.append(int(string[start:start+size]))
        start += size

    # Convertir la liste de nombres non signés en vecteur numpy
    vector = np.array(unsigned_vector, dtype=np.int8)

    return vector

## (a). $Codage$

In [44]:
def encoding_huffman(strings):
    huffman_tree_result = huffman_tree(strings)  # Stockez l'arbre de Huffman dans une variable distincte
    huffmanCode = huffman_dictionnary_code(huffman_tree_result)
    compressed_string = ''
    
    for char in strings:
        compressed_string += huffmanCode[char]
        
    return compressed_string, huffman_tree_result


## (b). $Décodage$

In [45]:
def decoding_huffman(compressed_string, huffman_tree):
    decoded_string = ''
    current_node = huffman_tree
    
    for bit in compressed_string:
        if bit == '0':
            current_node = current_node[0]
        else:
            current_node = current_node[1]
        
        if isinstance(current_node, str):
            decoded_string += current_node
            current_node = huffman_tree
    
    return decoded_string

## (-). $Fonctions$ $de$ $codage$ $du$ $vecteur$ $à$ $Huffman$

### $Codage$

In [49]:
def huffman_concatenated(vect):
  string, vect_lengths = convert_and_concatenate(vect)
  huffman_code, huffman_tr = encoding_huffman(string)

  return huffman_code, huffman_tr, vect_lengths

### $Décodage$

In [50]:
def huffman_deconcatenated(huffman_code, huffman_tr, vect_lengths):
    decoded = decoding_huffman(huffman_code, huffman_tr)
    vect = inverse_convert_and_concatenate(decoded, vect_lengths)
    
    return vect

# (9). $Codage$ $par$ $LZW$

## (-). $Fonctions$ $intermediaires$

### $Fusion$ $de$ $la$ $liste$ $des$ $codes$ $Huffman$ $de$ $tous$ $les$ $blocs$

In [51]:
def fusionner(strings_list):
    tailles = []
    string_fusionned = ""
    
    # Parcourir tous les éléments de la liste
    for string in strings_list:
        # Convertir chaque nombre en chaîne de caractères et l'ajouter à la chaîne fusionnée
        string_fusionned += string
        
        # Ajouter la taille de l'élément à la liste des tailles
        tailles.append(len(string))
    
    # Retourner la chaîne de caractères fusionnée et la liste des tailles
    return string_fusionned, tailles

### $Séparation$ $de$ $la$ $liste$ $des$ $codes$ $Huffman$ $de$ $tous$ $les$ $blocs$

In [52]:
def separer(string_fusionned, tailles):
    strings_list = []
    debut = 0
    
    # Parcourir toutes les tailles dans la liste
    for taille in tailles:
        # Extraire le sous-chaîne correspondant à la taille actuelle
        sous_chaine = string_fusionned[debut:debut+taille]
        
        # Convertir la sous-chaîne en nombre et l'ajouter à la liste des nombres
        strings_list.append(sous_chaine)
        
        # Mettre à jour la position de départ pour la prochaine sous-chaîne
        debut += taille
    
    # Retourner la liste des nombres
    return strings_list

## (a). $Codage$

In [53]:
def encoding_lzw(data, color_palette):
    alphabet = [str(i) for i in range(len(color_palette))]
    data_fused, taille = fusionner(data)  # Fusionner les chaînes de caractères
    encoded_data = []

    dictionary = {}  # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[char] = i

    prefix = ''
    for char in data_fused:
        new_entry = prefix + char
        if new_entry in dictionary:
            prefix = new_entry
        else:
            encoded_data.append(dictionary[prefix])
            dictionary[new_entry] = len(dictionary)
            prefix = char

    if prefix:
        encoded_data.append(dictionary[prefix])

    return encoded_data, alphabet, taille

## (b). $Décodage$

In [54]:
def decoding_lzw(compressed_data, alphabet, tailleL):
    result = []
    dictionary = {}
    current_code = len(alphabet)

    # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[i] = char

    old_entry = dictionary[compressed_data[0]]
    result.append(old_entry)
    for new_entry in compressed_data[1:]:
        if new_entry in dictionary:
            entry = dictionary[new_entry]
        elif new_entry == current_code:
            entry = old_entry + old_entry[0]
        else:
            raise ValueError("Mauvaise séquence compressée")

        result.append(entry)

        # Utilisez le même dictionnaire pour la décompression
        dictionary[current_code] = old_entry + entry[0]
        current_code += 1
        old_entry = entry

    result = ''.join(result)
    result = separer(result, tailleL)
    
    return result

## $Fonction$ $intermediaire$ $pour$ $la$ $compression$

In [55]:
def intermediaireC(imageA, num_colorsA):
  color_paletteA, palette_indicesA = kmeans_clustering_palette(imageA, num_colorsA)
  mapped_indicesA = map_to_palette(imageA, color_paletteA)
  blocsA, shapeTA = subdivision2D(mapped_indicesA, num_blocs)
  dataA = []
  huffman_trsA = []
  vect_lengthssA = []
  
  for blocA in blocsA:
    result1A = sous_echantillonnage_4_2_0(blocA)
    dct_matrixA = apply_dct(result1A)
    result11A, mean_valueA = quantization_mean(dct_matrixA)
    ligneA, shapetA = ligne_scan(result11A)
    huffman_codeA, huffman_trA, vect_lengthsA = huffman_concatenated(ligneA)
    
    if not huffman_codeA:
        print("La sortie de la fonction Huffman est vide.")
        print(ligneA)
        print(result11A)
        
    huffman_trsA.append(huffman_trA)
    vect_lengthssA.append(vect_lengthsA)
    dataA.append(huffman_codeA)
  
  encoded_dataA, alphabetA, tailleA = encoding_lzw(dataA, color_paletteA)
    
  return encoded_dataA, alphabetA, tailleA, huffman_trsA, vect_lengthssA, shapeTA, shapetA, mean_valueA, mapped_indicesA, color_paletteA, palette_indicesA

## $Fonction$ $intermediaire$ $pour$ $la$ $décompression$

In [56]:
def intermediaireD(encoded_dataB, alphabetB, tailleB, huffman_trB, vect_lengthsB, shapeTB, shapetB, mean_valueB, mapped_indicesB, color_paletteB, palette_indicesB):
  resultB = decoding_lzw(encoded_dataB, alphabetB, tailleB)
  codesB = []

  for i in range(len(resultB)):
    vect1B = huffman_deconcatenated(resultB[i], huffman_trB[i], vect_lengthsB[i])
    matB = inverse_ligne_scan(vect1B, shapetB)
    quantB = dequantization_mean(matB, mean_valueB)
    dctB = inverse_dct(quantB)
    subsapB = inverse_sous_echantillonnage_4_2_0(dctB)
    codesB.append(subsapB)
  
  code7 = reconstitution2D(codesB, shapeTB)
  mapp = inverse_map_to_palette(mapped_indicesB, color_paletteB)
  imageB = inverse_kmeans_clustering_palette(color_paletteB, palette_indicesB)
  return imageB

## $Fonction$ $intermediaire$ $pour$ $la$ $création$ $de$ $la$ $partie$ $"header"$

In [57]:
def create_header(path, num_colors):
  image = Image.open(path)
  return intermediaireC(image, num_colors)[1:]

## $Création$ $de$ $la$ $partie$ $"data"$ $(donnée)$

In [58]:
def data_irm(path, num_colors):
  image = Image.open(path)
  return intermediaireC(image, num_colors)[0]

## $Création$ $de$ $la$ $partie$ $"header"$ $(entête)$

In [59]:
def header_irm(path, num_colors):
    alphabet, taille, huffman_trT, vect_lengthsT, shapeT, shapet, mean_value, mapped_indices, color_palette, palette_indices = create_header(path, num_colors)
    header = f"alphabet: {alphabet}\n"
    header += f"taille: {taille}\n"
    header += f"huffman_trT: {huffman_trT}\n"
    header += f"vect_lengthsT: {vect_lengthsT}\n"
    header += f"shapeT: {shapeT}\n"
    header += f"shapet: {shapet}\n"
    header += f"mean_value: {mean_value}\n"
    header += f"mapped_indices: {list(map(list,list(mapped_indices)))}\n"
    header += f"color_palette: {list(map(list,list(color_palette)))}\n"
    header += f"palette_indices: {list(map(list,list(palette_indices)))}\n"
    return header

## $Fonction$ $de$ $compression$

In [60]:
def compression_irm(path1, num_colors, path2):
    header = header_irm(path1, num_colors)
    data = data_irm(path1, num_colors)
    compressed_data = "\n".join(str(nombre) for nombre in data)
    with open(path2, 'w') as f:
        f.write(header)
        f.write(compressed_data)

## $Fonction$ $de$ $décompression$

In [61]:
def decompression_irm(path1, path2):
    with open(path1, 'r') as f:
        lines = f.readlines()
        numeric_lines = [line.strip() for line in lines if re.match(r'^\d+$', line.strip())]
        encoded_data = [int(nombre.strip()) for nombre in numeric_lines]
        alphabet = np.array(eval(lines[0].split(': ')[1]))
        taille = np.array(eval(lines[1].split(': ')[1]))
        huffman_trT = eval(lines[2].split(': ')[1])
        vect_lengthsT = eval(lines[3].split(': ')[1])
        shapeT = eval(lines[4].split(': ')[1])
        shapet = eval(lines[5].split(': ')[1])
        mean_value = eval(lines[6].split(': ')[1]) 
        mapped_indices = np.array(eval(lines[7].split(': ')[1]))
        color_palette = np.array(eval(lines[8].split(': ')[1]))
        palette_indices = np.array(eval(lines[9].split(': ')[1]))
    
    image = intermediaireD(encoded_data, alphabet, taille, huffman_trT, vect_lengthsT, shapeT, shapet, mean_value, mapped_indices, color_palette, palette_indices)
    image.save(path2)
    return image

# $APPLICATION$ $COMPLÈTE$

## $Compression$

In [62]:
compression_irm(path, num_colors, path1)

/home/issa-sangare/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

In [ ]:
result1 = data_irm(path, num_colors)
result = header_irm(path, num_colors)
img = intermediaireD(result1, result[0], result[1], result[2], result[3], result[4], result[5], result[6], result[7], result[8], result[9])

## $Décompression$

In [ ]:
decompression_irm(path1, path2)

ValueError: invalid literal for int() with base 10: ''

## $Affichage$

In [ ]:
img1 = Image.open(path)
img2 = Image.open(path2)

In [1]:
plot_images(img1, img2)

NameError: name 'plot_images' is not defined

## $Mean$ $Square$ $Error$ $(MSE):$ $Erreur$ $Quadratique$ $Moyenne$

In [ ]:
erreur = MSE(img1, img2)
print(f"L'erreur quadratique moyenne (MSE) est: {erreur}")

## $Taux$ $de$ $compression$

In [ ]:
taux = taux_compression(path, path2)
print(f"Taux de compression: {100 * taux} %")